# Avoiding numerical pitfalls

## The harmonic series is convergent in floating point arithmetics

\begin{align*}
\sum_{n=1}^{\infty} \; \frac{1}{n} \quad = \quad 34.1220356680478715816207113675773143768310546875
\end{align*}

(usually it is famously known to diverge to $\infty$)

### References

Proof of divergence:
http://www.free-education-resources.com/www.mathematik.net/harmonische-reihen/hr1s20.htm

Proof of convergence for floating point:
http://www.maths.tcd.ie/pub/ims/bull71/recipnote.pdf

Resolution:
http://fredrik-j.blogspot.de/2009/02/how-not-to-compute-harmonic-numbers.html

## Basics

### References

http://www.johndcook.com/blog/2009/04/06/numbers-are-a-leaky-abstraction/

http://www.codeproject.com/Articles/29637/Five-Tips-for-Floating-Point-Programming

http://www.codeproject.com/Articles/25294/Avoiding-Overflow-Underflow-and-Loss-of-Precision

In floating point arithmetic, subtraction is rather inaccurate. Observe that 2-1.8 is not 0.2.
However, Python catches this well known phenomenon in its str-method and converts the output to a convenient number. The following two lines illustrate not only numeric subtaction inaccuracy, but also the difference between `repr` and `str`. `repr` is designed to represent the value accurately, while `str` is intended for a convenient output format.

In [7]:
print repr(2-1.8)
print str(2-1.8)

0.19999999999999996
0.2


Just to mention for completeness: Don't use exact equals-operator on floats:

In [20]:
print (2-1.8 == 0.2)
#Python-hack that actually works surprisingly well:
print (str(2-1.8) == str(0.2))

#Recommended method with control over matching-precision:
threshold = 0.000000001
print ((2-1.8) - 0.2 < threshold)

False
True
True


Let's solve a quadratic equation. Naive solving becomes bad if low and large coefficients occur.
Consider the equation $3 x^2 + 10^6 x + 5 = 0$

In [80]:
import numpy as np

a = 3.0
b = 10e7
c = 0.005

def f(x):
    return a*x**2+b*x+c

def solve_pq():
    p = b/a
    q = c/a
    D = (p/2.0)**2 - q
    r1 = -p/2.0+D**0.5
    r2 = -p/2.0-D**0.5
    return (r1, r2)

def solve_pq2():
    p = b/a
    q = c/a
    D = (p/2.0)**2 - q
    r1 = -2.0*q/(p+2.0*D**0.5)
    r2 = -p/2.0-D**0.5
    return (r1, r2)

def solve_companion():
    p = b/a
    q = c/a
    C = np.array([[0.0, -q], [1.0, -p]])
    return np.linalg.eigvals(C)

result = solve_pq()
print "pq"
print repr(result)
print repr(f(result[0]))
print repr(f(result[1]))

result = solve_pq2()
print "pq2"
print repr(result)
print repr(f(result[0]))
print repr(f(result[1]))

result = solve_companion()
print "companion"
print repr(result)
print repr(f(result[0]))
print repr(f(result[1]))


pq
(0.0, -33333333.333333332)
0.005
0.005
pq2
(-5.000000000000001e-11, -33333333.333333332)
-8.673617379884035e-19
0.005
companion
array([ -3.72529030e-09,  -3.33333333e+07])
-0.36752902984619135
0.0050000000000000001


## Don't invert that matrix

### References

http://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/

### Summary:

* There's hardly ever a good reason to invert a matrix
* Solve linear equation systems directly
* Apply a QR-decomposition to solve multiple systems with the same matrix (but different right side)
* Even if the inverse is given for free, direct solving is still more accurate
* Inverses of sparse matrices are in general dense

## Polynomes

* Horner's method for numerically stable evaluation
* NumPy's Polynome-class has this built-in

### For expansion:

* sometimes we use monomials to expand input data
* a subsequent algorithm is supposed to figure out the factors and assemble polynomials
* these will be unstable in a naive approach
* instead of monomials use a numerically stable base like legendre polynomials
* you might need to adjust them for your value-set